In [1]:
import polars as pl 
import numpy as np
from eda_utils import _f_score, mrmr, MRMR_STRATEGY
from sklearn.datasets import make_classification
from sklearn.feature_selection import f_regression
import pandas as pd
from time import perf_counter

In [2]:
# create some data
orig_x, orig_y = make_classification(n_samples = 1_000_000, n_features = 50, n_informative = 20, n_redundant = 30)

In [3]:
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
df_pd = df_pl.to_pandas()

In [4]:
def mrmr_my_rewrite(df:pl.DataFrame, target:str, k:int) -> list[str]:

    features = df.columns
    features.remove(target)

    start = perf_counter()
    f_scores = _f_score(df, target, features)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    df_scaled = df.select(features).with_columns(
        (pl.col(f) - pl.col(f).mean())/pl.col(f).std() for f in features
    )

    cumulating_sum = np.zeros(len(features)) # For each feature at index i, we keep a cumulating sum
    top_idx = np.argmax(f_scores)
    selected_features = [features[top_idx]]
    for j in range(1, k): 
        argmax = -1
        current_max = -1
        last_selected = selected_features[-1]
        for i,f in enumerate(features):
            if f not in selected_features:
                # Left = cumulating sum of abs corr
                # Right = abs correlation btw last_selected and f
                cumulating_sum[i] += np.abs((df_scaled.get_column(last_selected)*df_scaled.get_column(f)).mean())
                denominator = cumulating_sum[i] / j
                new_score = f_scores[i] / denominator
                if new_score > current_max:
                    current_max = new_score
                    argmax = i

        selected_features.append(features[argmax])

    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected_features
    

In [5]:
def mrmr_np_rewrite(df:pl.DataFrame, target:str, k:int) -> list[str]:

    features = df.columns
    features.remove(target)

    start = perf_counter()
    f_scores = _f_score(df, target, features)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    x = df.select(features).to_numpy()
    x_scaled = (x - x.mean(axis=0)) / x.std(axis=0)

    cumulating_sum = np.zeros(len(features)) # For each feature at index i, we keep a cumulating sum
    top_idx = np.argmax(f_scores)
    selected_features = [features[top_idx]]
    last_selected_idx = top_idx
    for j in range(1, k): 
        argmax = -1
        current_max = -1
        for i,f in enumerate(features):
            if f not in selected_features:
                # Left = cumulating sum of abs corr
                # Right = abs correlation btw last_selected and f
                cumulating_sum[i] += np.abs(np.mean(x_scaled[:,last_selected_idx]*x_scaled[:,i]))
                denominator = cumulating_sum[i] / j
                new_score = f_scores[i] / denominator
                if new_score > current_max:
                    current_max = new_score
                    argmax = i

        selected_features.append(features[argmax])
        last_selected_idx = argmax

    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected_features

In [6]:
def mrmr_medium(df:pd.DataFrame, target:str, k:int) -> list[str]:

    features = list(df.columns)
    features.remove(target)

    X = df[features]
    y = df[target]

    # compute F-statistics and initialize correlation matrix
    start = perf_counter()
    F = pd.Series(f_regression(X, y)[0], index = X.columns)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

    # initialize list of selected features and list of excluded features
    selected = []
    not_selected = X.columns.to_list()

    # repeat K times
    for i in range(k):
    
        # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
            
        # compute FCQ score for all the (currently) excluded features (this is Formula 2)
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        
        # find best feature, add it to selected and remove it from not_selected
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)

    # print(selected) 
    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected 

In [7]:
mrmr_my_rewrite(df_pl, "target", 20)

Spent 0.07s to compute f score.
Took 0.96s to compute mrmr.


['column_41',
 'column_29',
 'column_36',
 'column_2',
 'column_18',
 'column_6',
 'column_26',
 'column_44',
 'column_9',
 'column_47',
 'column_12',
 'column_22',
 'column_42',
 'column_37',
 'column_40',
 'column_19',
 'column_4',
 'column_15',
 'column_13',
 'column_27']

In [8]:
# Use my version of f_score computation.
# Use pure NumPy for the rest of the computation.
# This shows that copying to NumPy, does not pay off. Stay in Polars is a better idea.
mrmr_np_rewrite(df_pl, "target", 20)

Spent 0.07s to compute f score.
Took 2.57s to compute mrmr.


['column_41',
 'column_29',
 'column_36',
 'column_2',
 'column_18',
 'column_6',
 'column_26',
 'column_44',
 'column_9',
 'column_47',
 'column_12',
 'column_22',
 'column_42',
 'column_37',
 'column_40',
 'column_19',
 'column_4',
 'column_15',
 'column_13',
 'column_27']

In [9]:
mrmr_medium(df_pd, "target", 20)

Spent 0.15s to compute f score.
Took 9.81s to compute mrmr.


['column_41',
 'column_29',
 'column_36',
 'column_2',
 'column_18',
 'column_6',
 'column_26',
 'column_44',
 'column_9',
 'column_47',
 'column_12',
 'column_22',
 'column_42',
 'column_37',
 'column_40',
 'column_19',
 'column_4',
 'column_15',
 'column_13',
 'column_27']

In [10]:
from mrmr import mrmr_classif

In [11]:
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    
    features = list(df.columns)
    features.remove(target)

    X = df[features]
    y = df[target]

    # use mrmr classification
    return mrmr_classif(X, y, K = k)

In [12]:
mrmr_package(df_pd, "target", 20)

100%|██████████| 20/20 [00:12<00:00,  1.62it/s]


['column_41',
 'column_29',
 'column_36',
 'column_2',
 'column_18',
 'column_6',
 'column_26',
 'column_44',
 'column_9',
 'column_47',
 'column_12',
 'column_22',
 'column_42',
 'column_37',
 'column_40',
 'column_19',
 'column_4',
 'column_15',
 'column_13',
 'column_27']

In [13]:
# MRMR from my repo 

mrmr(df_pl, target="target", k=20)

Top 5 feature importance is (by MRMR_STRATEGY.F_SCORE):
shape: (5, 2)
┌───────────┬───────────────────────┐
│ feature   ┆ MRMR_STRATEGY.F_SCORE │
│ ---       ┆ ---                   │
│ str       ┆ f64                   │
╞═══════════╪═══════════════════════╡
│ column_41 ┆ 155212.46148          │
│ column_2  ┆ 145665.316367         │
│ column_6  ┆ 129582.640269         │
│ column_26 ┆ 102993.389405         │
│ column_44 ┆ 83028.443679          │
└───────────┴───────────────────────┘
Found 1st feature by MRMR: column_41. 1/20
Found 2th feature by MRMR: column_29. 2/20
Found 3th feature by MRMR: column_36. 3/20
Found 4th feature by MRMR: column_2. 4/20
Found 5th feature by MRMR: column_18. 5/20
Found 6th feature by MRMR: column_6. 6/20
Found 7th feature by MRMR: column_26. 7/20
Found 8th feature by MRMR: column_44. 8/20
Found 9th feature by MRMR: column_9. 9/20
Found 10th feature by MRMR: column_47. 10/20
Found 11th feature by MRMR: column_12. 11/20
Found 12th feature by MRMR: column_22.

mrmr_rank,feature
i64,str
1,"""column_41"""
2,"""column_29"""
3,"""column_36"""
4,"""column_2"""
5,"""column_18"""
6,"""column_6"""
7,"""column_26"""
8,"""column_44"""
9,"""column_9"""


In [14]:
mrmr(df_pl, target="target", k=20, strategy=MRMR_STRATEGY.RF, params={"n_estimators":20, "max_depth":5, "n_jobs":-1})

Random forest is not deterministic by default. Results may vary.
Top 5 feature importance is (by MRMR_STRATEGY.RF):
shape: (5, 2)
┌───────────┬──────────────────┐
│ feature   ┆ MRMR_STRATEGY.RF │
│ ---       ┆ ---              │
│ str       ┆ f64              │
╞═══════════╪══════════════════╡
│ column_6  ┆ 0.119616         │
│ column_2  ┆ 0.115604         │
│ column_41 ┆ 0.090616         │
│ column_26 ┆ 0.083756         │
│ column_12 ┆ 0.044039         │
└───────────┴──────────────────┘
Found 1st feature by MRMR: column_6. 1/20
Found 2th feature by MRMR: column_29. 2/20
Found 3th feature by MRMR: column_2. 3/20
Found 4th feature by MRMR: column_41. 4/20
Found 5th feature by MRMR: column_26. 5/20
Found 6th feature by MRMR: column_9. 6/20
Found 7th feature by MRMR: column_37. 7/20
Found 8th feature by MRMR: column_47. 8/20
Found 9th feature by MRMR: column_22. 9/20
Found 10th feature by MRMR: column_35. 10/20
Found 11th feature by MRMR: column_12. 11/20
Found 12th feature by MRMR: colum

mrmr_rank,feature
i64,str
1,"""column_6"""
2,"""column_29"""
3,"""column_2"""
4,"""column_41"""
5,"""column_26"""
6,"""column_9"""
7,"""column_37"""
8,"""column_47"""
9,"""column_22"""


In [15]:
mrmr(df_pl, target="target", k=20, strategy=MRMR_STRATEGY.XGB, params={"n_estimators":20, "max_depth":5})

XGB is not deterministic by default. Results may vary.
Top 5 feature importance is (by MRMR_STRATEGY.XGB):
shape: (5, 2)
┌───────────┬───────────────────┐
│ feature   ┆ MRMR_STRATEGY.XGB │
│ ---       ┆ ---               │
│ str       ┆ f64               │
╞═══════════╪═══════════════════╡
│ column_6  ┆ 0.067482          │
│ column_41 ┆ 0.05943           │
│ column_2  ┆ 0.054631          │
│ column_29 ┆ 0.049705          │
│ column_22 ┆ 0.047165          │
└───────────┴───────────────────┘
Found 1st feature by MRMR: column_6. 1/20
Found 2th feature by MRMR: column_29. 2/20
Found 3th feature by MRMR: column_9. 3/20
Found 4th feature by MRMR: column_2. 4/20
Found 5th feature by MRMR: column_41. 5/20
Found 6th feature by MRMR: column_10. 6/20
Found 7th feature by MRMR: column_36. 7/20
Found 8th feature by MRMR: column_47. 8/20
Found 9th feature by MRMR: column_16. 9/20
Found 10th feature by MRMR: column_37. 10/20
Found 11th feature by MRMR: column_5. 11/20
Found 12th feature by MRMR: colu

mrmr_rank,feature
i64,str
1,"""column_6"""
2,"""column_29"""
3,"""column_9"""
4,"""column_2"""
5,"""column_41"""
6,"""column_10"""
7,"""column_36"""
8,"""column_47"""
9,"""column_16"""
